In [1]:
#import models to be used
import numpy as np
import pandas as pd
import librosa
import multiprocessing
import sys
import os

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras import utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard

from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils

from scipy.io import wavfile
from matplotlib import pyplot as plt
import glob 
import librosa.display

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

Using TensorFlow backend.


In [2]:
import wave
path = 'call/'
filenames = []
label = []
for filename in glob.glob(os.path.join(path, '*.wav')):
    filenames.append(filename)
    if "english" in filename:
        label.append(1)
    if "mandarin" in filename:
        label.append(0)

In [4]:
call = pd.DataFrame({'feature': filenames, 'label': label})

In [5]:
#we extract mfcc feature from data
def extract_feature(file_name):
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None
    return (mfccs)

feature = []
for i in range(len(filenames)):
    feature.append(extract_feature(filenames[i]))
     

temp = pd.DataFrame({'feature': feature, 'label': label})

In [6]:
# prepare data for building model and split data into train data and validation data

X = np.array(temp.feature.tolist())
y = np.array(temp.label.tolist())

lb = LabelEncoder()

y = utils.to_categorical(lb.fit_transform(y))

cut_x = round(0.8*len(X))
cut_y = round(0.8*len(y))

valX = X[len(X) - cut_x:]
valY = y[len(y) - cut_y:]


X = X[0:len(X) - cut_x]
y = y[0:len(y) - cut_y]

In [7]:
# build model
labels = y.shape[1]
filter_size = 2


model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
model.fit(X, y, batch_size=32, epochs=5, validation_data=(valX, valY))

Instructions for updating:
Use tf.cast instead.
Train on 151 samples, validate on 605 samples
Epoch 1/5
151/151 [==============================] - 0s - loss: 4.5054 - acc: 0.7152 - val_loss: 2.3977 - val_acc: 0.8512
Epoch 2/5
151/151 [==============================] - 0s - loss: 4.4832 - acc: 0.7219 - val_loss: 2.3977 - val_acc: 0.8512
Epoch 3/5
151/151 [==============================] - 0s - loss: 4.4832 - acc: 0.7219 - val_loss: 2.3977 - val_acc: 0.8512
Epoch 4/5
151/151 [==============================] - 0s - loss: 4.4832 - acc: 0.7219 - val_loss: 2.3977 - val_acc: 0.8512
Epoch 5/5
151/151 [==============================] - 0s - loss: 4.4832 - acc: 0.7219 - val_loss: 2.3977 - val_acc: 0.8512


In [9]:
from keras.models import load_model
model.save('call.h5')